<a href="https://colab.research.google.com/github/Afrizal2609/Hate-Speech-Detection-of-banjar-Languange/blob/main/Colab/hate%20speech%20classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loading Library**

In [1]:
pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.2 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [9]:
#Library
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
import spacy

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
import gensim
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

from sklearn import metrics
from sklearn.metrics import f1_score

!python -m spacy download en_core_web_md
nlp = spacy.load("en_core_web_md")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 1.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


# **Loading Dataset**

In [3]:
# Upload Dataset
#load dataset
df = pd.read_csv('/content/banjar.csv')

# preview of dataset
df.head()

,text,Label
0,bayangkan kalau kadada cctv warga nang dijebak...,0
1,sagan apa naik pangkat,0
2,ini hanyar nang ketahuan balum lagi nang kada ...,0
3,handak komen tapi takutan jadi target mbah ni,0
4,Kaini banar menyasah target nya oknum oknum,0


# **Preparing Dataset**

In [4]:
# rename columns
df.columns = ['Text', 'Label']

# remove missing values
df = df.dropna()

# encode target label
le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])

# establish input and output
X = list(df['Text'])
y = list(df['Label'])



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [5]:
#Custom Class for GloveVectorizer
class GloveVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, model_name="en_core_web_md"):
        self._nlp = spacy.load(model_name)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.concatenate(
            [self._nlp(doc).vector.reshape(1, -1) for doc in X]
        )

# **Naive Bayes**


**Naive Bayes with Word N-Gram**

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cv = CountVectorizer(analyzer = 'word',ngram_range=(1,1), stop_words='english')
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [27]:
clf = MultinomialNB()
clf.fit(X_train_cv, y_train)
# create predictions
y_pred = clf.predict(X_test_cv)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

test_text=cv.transform(["bungul siapa nyawa ngini"])

print("\n")
print("Prediction:")
print(clf.predict(test_text))

F-1 score : 0.9064
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      2697
           1       0.81      0.32      0.46       380

    accuracy                           0.91      3077
   macro avg       0.86      0.65      0.70      3077
weighted avg       0.90      0.91      0.89      3077

confusion matrix:
[[2668   29]
 [ 259  121]]


Prediction:
[1]


**Naive Bayes with TF-IDF**

In [40]:
#TFIDF
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tfidf = TfidfVectorizer()
tfidf_train = tfidf.fit_transform(X_train)
#Fitting and transforming input data
tfidf_train = tfidf.transform(X_train)
tfidf_test = tfidf.transform(X_test)

In [41]:
clf = MultinomialNB()
clf.fit(tfidf_train, y_train)
# create predictions
y_pred = clf.predict(tfidf_test)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

test_text=tfidf.transform(["bungul siapa nyawa ngini"])

print("\n")
print("Prediction:")
print(clf.predict(test_text))

F-1 score : 0.896
              precision    recall  f1-score   support

           0       0.89      1.00      0.94      2697
           1       1.00      0.16      0.27       380

    accuracy                           0.90      3077
   macro avg       0.95      0.58      0.61      3077
weighted avg       0.91      0.90      0.86      3077

confusion matrix:
[[2697    0]
 [ 320   60]]


Prediction:
[0]


**Naive Bayes with Word N-Gram and TF-IDF**

In [43]:
Tfidf = TfidfVectorizer(min_df=5, ngram_range=(1, 1))
tfidf_features = Tfidf.fit_transform(df.Text)

X_train, X_test, y_train, y_test = train_test_split(tfidf_features, df["Label"], test_size = 1/5, random_state = 50)

In [45]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
# create predictions
y_pred = clf.predict(X_test)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

test_text=Tfidf.transform(["bungul siapa nyawa ngini"])

print("\n")
print("Prediction:")
print(clf.predict(test_text))

F-1 score : 0.8963
              precision    recall  f1-score   support

           0       0.89      1.00      0.94      2657
           1       0.95      0.25      0.40       420

    accuracy                           0.90      3077
   macro avg       0.92      0.63      0.67      3077
weighted avg       0.90      0.90      0.87      3077

confusion matrix:
[[2651    6]
 [ 313  107]]


Prediction:
[1]


**Naive Bayes with Word2Vec**

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
w2v_model = gensim.models.Word2Vec(X_train, vector_size=100, window=5, min_count=2)

words = set(w2v_model.wv.index_to_key )
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [48]:
clf = GaussianNB()
clf.fit(X_train_vect_avg, y_train)
# create predictions
y_pred = clf.predict(X_test_vect_avg)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

test_text=(["tambuk siapa nyawa ngini"])
test_text_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in test_text])
test_text_avg = []
for v in test_text_vect:
    if v.size:
        test_text_avg.append(v.mean(axis=0))
    else:
        test_text_avg.append(np.zeros(100, dtype=float))

print("\n")
print("Prediction:")
print(clf.predict(test_text_avg))

F-1 score : 0.7764
              precision    recall  f1-score   support

           0       0.90      0.84      0.87      2697
           1       0.22      0.33      0.27       380

    accuracy                           0.78      3077
   macro avg       0.56      0.58      0.57      3077
weighted avg       0.82      0.78      0.79      3077

confusion matrix:
[[2264  433]
 [ 255  125]]


Prediction:
[0]


**Naive Bayes with Glove**

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

g = GloveVectorizer()
nb = GaussianNB()

pipeline = make_pipeline(g, nb)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

test_text=(["siapa nyawa ngini"])

print("\n")
print("Prediction:")
print(pipeline.predict(test_text))

F-1 score : 0.416
              precision    recall  f1-score   support

           0       0.91      0.37      0.53      2697
           1       0.14      0.74      0.24       380

    accuracy                           0.42      3077
   macro avg       0.53      0.55      0.38      3077
weighted avg       0.81      0.42      0.49      3077

confusion matrix:
[[1000 1697]
 [ 100  280]]


Prediction:
[1]


# **SVM**


**SVM with Word N-Gram**

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cv = CountVectorizer(analyzer = 'word',ngram_range=(1,1), stop_words='english')
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [51]:
svc = LinearSVC()
svc.fit(X_train_cv, y_train)
# create predictions
y_pred = svc.predict(X_test_cv)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

test_text=cv.transform(["tambuk siapa nyawa ngini"])

print("\n")
print("Prediction:")
print(svc.predict(test_text))

F-1 score : 0.8983
              precision    recall  f1-score   support

           0       0.93      0.96      0.94      2697
           1       0.61      0.48      0.54       380

    accuracy                           0.90      3077
   macro avg       0.77      0.72      0.74      3077
weighted avg       0.89      0.90      0.89      3077

confusion matrix:
[[2582  115]
 [ 198  182]]


Prediction:
[1]


**SVM with TF-IDF**

In [65]:
#TFIDF
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tfidf = TfidfVectorizer()
tfidf_train = tfidf.fit_transform(X_train)
#Fitting and transforming input data
tfidf_train = tfidf.transform(X_train)
tfidf_test = tfidf.transform(X_test)

In [72]:
# train svm
svc = LinearSVC()
svc.fit(tfidf_train, y_train)#TFIDF

y_pred = svc.predict(tfidf_test)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

test_text=tfidf.transform(["tambuk siapa nyawa ngini"])

print("\n")
print("Prediction:")
print(svc.predict(test_text))


F-1 score : 0.9106
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      2697
           1       0.75      0.42      0.54       380

    accuracy                           0.91      3077
   macro avg       0.83      0.70      0.74      3077
weighted avg       0.90      0.91      0.90      3077

confusion matrix:
[[2643   54]
 [ 221  159]]


Prediction:
[1]


**SVM with Word N-Gram and TF-IDF**

In [15]:
Tfidf = TfidfVectorizer(min_df=5, ngram_range=(1, 1))
tfidf_features = Tfidf.fit_transform(df['Text'])

X_train, X_test, y_train, y_test = train_test_split(tfidf_features, df["Label"], test_size = 1/5, random_state = 50)

In [16]:
# train svm
svc = LinearSVC()
svc.fit(X_train, y_train)#TFIDF

y_pred = svc.predict(X_test)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

test_text=Tfidf.transform(["bungul siapa nyawa ngini"])

print("\n")
print("Prediction:")
print(svc.predict(test_text))

F-1 score : 0.9032
              precision    recall  f1-score   support

           0       0.91      0.98      0.95      2657
           1       0.79      0.40      0.53       420

    accuracy                           0.90      3077
   macro avg       0.85      0.69      0.74      3077
weighted avg       0.89      0.90      0.89      3077

confusion matrix:
[[2613   44]
 [ 254  166]]


Prediction:
[1]


**SVM with Word2Vec**

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Label'], test_size=0.2, random_state=42)
w2v_model = gensim.models.Word2Vec(X_train, vector_size=100, window=5, min_count=5, workers=4)

w2v_model.wv.index_to_key

words = set(w2v_model.wv.index_to_key)
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.


In [14]:
# train svm
svc = LinearSVC()
svc.fit(X_train_vect_avg, y_train.values)

y_pred = svc.predict(X_test_vect_avg)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

test_text=(["tambuk siapa nyawa ngini"])
test_text_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in test_text])
test_text_avg = []
for v in test_text_vect:
    if v.size:
        test_text_avg.append(v.mean(axis=0))
    else:
        test_text_avg.append(np.zeros(100, dtype=float))

print("\n")
print("Prediction:")
print(svc.predict(test_text_avg))

F-1 score : 0.8804
              precision    recall  f1-score   support

           0       0.88      1.00      0.94      2697
           1       0.93      0.03      0.07       380

    accuracy                           0.88      3077
   macro avg       0.90      0.52      0.50      3077
weighted avg       0.89      0.88      0.83      3077

confusion matrix:
[[2696    1]
 [ 367   13]]


Prediction:
[0]


**SVM with Glove**

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

g = GloveVectorizer()
svm = LinearSVC()

pipeline = make_pipeline(g, svm)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")

print(metrics.confusion_matrix(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


F-1 score : 0.8729
              precision    recall  f1-score   support

           0       0.88      0.99      0.93      2697
           1       0.33      0.02      0.04       380
           2       0.00      0.00      0.00         0

    accuracy                           0.87      3077
   macro avg       0.40      0.34      0.32      3077
weighted avg       0.81      0.87      0.82      3077

confusion matrix:
[[2678   16    3]
 [ 372    8    0]
 [   0    0    0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **Decision Tree**


**Decision Tree with word N-Gram**

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cv = CountVectorizer(analyzer = 'word',ngram_range=(1,5), stop_words='english')
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [56]:
dt = DecisionTreeClassifier()
dt.fit(X_train_cv, y_train)
# create predictions
y_pred = dt.predict(X_test_cv)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

test_text=cv.transform(["tambuk siapa nyawa ngini"])

print("\n")
print("Prediction:")
print(dt.predict(test_text))

F-1 score : 0.8947
              precision    recall  f1-score   support

           0       0.93      0.96      0.94      2697
           1       0.59      0.46      0.52       380

    accuracy                           0.89      3077
   macro avg       0.76      0.71      0.73      3077
weighted avg       0.89      0.89      0.89      3077

confusion matrix:
[[2577  120]
 [ 204  176]]


Prediction:
[1]


**Decision Tree with TF-IDF**

In [57]:
#TFIDF
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tfidf = TfidfVectorizer()
tfidf_train = tfidf.fit_transform(X_train)
#Fitting and transforming input data
tfidf_train = tfidf.transform(X_train)
tfidf_test = tfidf.transform(X_test)

In [59]:
dt = DecisionTreeClassifier()
dt.fit(tfidf_train, y_train)
# create predictions
y_pred = dt.predict(tfidf_test)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

test_text=tfidf.transform(["tambuk siapa nyawa ngini"])

print("\n")
print("Prediction:")
print(dt.predict(test_text))

F-1 score : 0.8833
              precision    recall  f1-score   support

           0       0.93      0.94      0.93      2697
           1       0.53      0.46      0.50       380

    accuracy                           0.88      3077
   macro avg       0.73      0.70      0.71      3077
weighted avg       0.88      0.88      0.88      3077

confusion matrix:
[[2542  155]
 [ 204  176]]


Prediction:
[1]


**Decision Tree with Word N-Gram and TF-IDF**

In [62]:
Tfidf = TfidfVectorizer(min_df=5, ngram_range=(1, 5))
tfidf_features = Tfidf.fit_transform(df.Text)

X_train, X_test, y_train, y_test = train_test_split(tfidf_features, df["Label"], test_size = 1/5, random_state = 50)

In [64]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
# create predictions
y_pred = dt.predict(X_test)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

test_text=tfidf.fit_transform(["tambuk siapa nyawa ngini"])

print("\n")
print("Prediction:")
print(dt.predict(test_text))

F-1 score : 0.8707
              precision    recall  f1-score   support

           0       0.92      0.93      0.93      2657
           1       0.53      0.49      0.51       420

    accuracy                           0.87      3077
   macro avg       0.72      0.71      0.72      3077
weighted avg       0.87      0.87      0.87      3077

confusion matrix:
[[2475  182]
 [ 216  204]]


Prediction:


ValueError: ignored

**Decision Tree with Word2Vec**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
w2v_model = gensim.models.Word2Vec(X_train, vector_size=100, window=5, min_count=2)

words = set(w2v_model.wv.index_to_key )
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train_vect_avg, y_train)

y_pred = dt.predict(X_test_vect_avg)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

F-1 score : 0.7751
              precision    recall  f1-score   support

           0       0.89      0.85      0.87      2697
           1       0.19      0.24      0.21       380

    accuracy                           0.78      3077
   macro avg       0.54      0.55      0.54      3077
weighted avg       0.80      0.78      0.79      3077

confusion matrix:
[[2293  404]
 [ 288   92]]


**Decision Tree with Glove**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

g = GloveVectorizer()
dt = DecisionTreeClassifier()

pipeline = make_pipeline(g, dt)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

print(metrics.classification_report(y_test, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

F-1 score : 0.7888
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      2697
           1       0.16      0.17      0.16       380

    accuracy                           0.79      3077
   macro avg       0.52      0.52      0.52      3077
weighted avg       0.79      0.79      0.79      3077

confusion matrix:
[[2363  334]
 [ 316   64]]
